# Molecular Dynamics Simulations of Protein in Water

In [ ]:
#@title 🚀 Install OpenMM & PDBFixer via conda
!pip install -q condacolab
import condacolab
condacolab.install()

# Now install packages via conda-forge
!conda install -q -y -c conda-forge openmm pdbfixer mdtraj py3Dmol mdanalysis

#!pip install mdanalysis

In [ ]:
#@title 📦 2. Import Python packages
import MDAnalysis as mda
from openmm.app import *
from openmm import *
from openmm.unit import *
from pdbfixer import PDBFixer
from sys import stdout
import py3Dmol
import mdtraj as md
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")

import py3Dmol
from openmm.app import PDBFile, Modeller
from pdbfixer import PDBFixer
from io import StringIO

def convert_dcd_to_pdb(topology_file, trajectory_file, output_pdb="trajectory.pdb"):
    """
    Convert a DCD trajectory to a multi-frame PDB using MDAnalysis.

    Parameters:
    - topology_file: e.g., "md_last.pdb"
    - trajectory_file: e.g., "md.dcd"
    - output_pdb: name of the output multi-model PDB
    """
    u = mda.Universe(topology_file, trajectory_file)
    with mda.Writer(output_pdb, u.atoms.n_atoms) as writer:
        for ts in u.trajectory:
            writer.write(u.atoms)

    print(f"✅ Saved trajectory to {output_pdb}")
    
def vmd(mol, width=600, height=400, zoom=True):
    """
    Visualize a structure using py3Dmol with enhanced style:
    - Protein as cartoon
    - Waters as lines
    - Ions as spheres
    """
    from openmm.app import PDBFile, Modeller
    from pdbfixer import PDBFixer
    from io import StringIO

    if isinstance(mol, (Modeller, PDBFile, PDBFixer)):
        buf = StringIO()
        PDBFile.writeFile(mol.topology, mol.positions, buf)
        pdb_str = buf.getvalue()
    elif isinstance(mol, str):
        with open(mol, 'r') as f:
            pdb_str = f.read()
    else:
        raise TypeError("Input must be a PDBFixer, Modeller, PDBFile, or filename")

    view = py3Dmol.view(width=width, height=height)
    view.addModel(pdb_str, 'pdb')

    # Protein
    view.setStyle({'resn': [
        'ALA','ARG','ASN','ASP','CYS','GLN','GLU','GLY','HIS','ILE',
        'LEU','LYS','MET','PHE','PRO','SER','THR','TRP','TYR','VAL'
    ]}, {'cartoon': {'color': 'spectrum'}})

    # Water
    view.addStyle({'resn': 'HOH'}, {'stick': {'color': 'blue'}})

    # Common ions (Na+, Cl-, K+, etc.)
    ion_residues = ['NA', 'CL', 'K', 'CA', 'MG', 'ZN']
    for ion in ion_residues:
        view.addStyle({'resn': ion}, {'sphere': {'color': 'grey', 'radius': 0.3}})

    if zoom:
        view.zoomTo()
    view.show()

def vmd_trajectory(pdb_file, width=600, height=400, interval=100):
    """
    Show multi-frame PDB trajectory as animation using py3Dmol.

    Parameters:
    - pdb_file: PDB file with multiple MODEL/ENDMDL blocks
    - interval: ms between frames
    """
    with open(pdb_file, 'r') as f:
        pdb_str = f.read()

    view = py3Dmol.view(width=width, height=height)
    view.addModelsAsFrames(pdb_str, 'pdb')
    view.setStyle({'cartoon': {'color': 'spectrum'}})
    view.animate({'loop': 'forward', 'interval': interval})
    view.zoomTo()
    view.show()

def load_protein(pdb_id: str = "1BFF", ph: float = 7.0) -> PDBFixer:
    """
    Loads and prepares a protein using PDBFixer.
    
    Steps:
    - Downloads structure from RCSB
    - Finds missing residues and atoms
    - Adds missing atoms and hydrogens
    
    Parameters:
        pdb_id: 4-character PDB ID
        ph: pH at which to add hydrogens
    
    Returns:
        PDBFixer object ready for modeling
    """
    fixer = PDBFixer(pdbid=pdb_id)
    fixer.findMissingResidues()
    fixer.findMissingAtoms()
    fixer.addMissingAtoms()
    fixer.addMissingHydrogens(pH=ph)
    return fixer

def gmx_pdb2gmx(protein, forcefield=None):
    """
    Prepare a Modeller system by removing nonstandard residues and adding water/ions.

    Parameters:
        protein: a PDBFixer object (already cleaned)
        forcefield: OpenMM ForceField (if None, uses Amber14 + TIP3P)
        padding: padding distance for solvation box
        ionic_strength: NaCl concentration for neutralization

    Returns:
        modeller: solvated, cleaned Modeller object
    """
    # Define the set of standard amino acid residue names
    standard_residues = {
        'ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS',
        'ILE', 'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP',
        'TYR', 'VAL'
    }
    
    # Initialize an OpenMM Modeller object with the protein's topology and coordinates
    modeller = Modeller(protein.topology, protein.positions)
    
    # Identify all residues in the structure that are NOT standard amino acids
    to_delete = [res for res in modeller.topology.residues()
                 if res.name not in standard_residues]
    
    # Remove non-standard residues (e.g., ligands, ions, cofactors)
    modeller.delete(to_delete)
    
    # Return the cleaned Modeller object (protein-only)
    return modeller


from openmm.app import Simulation
from openmm import Platform, LangevinIntegrator

# ✅ Function to build restraints on CA atoms
def build_ca_restraints(topology, positions, k=1000.0):
    force = CustomExternalForce("0.5 * k * ((x - x0)^2 + (y - y0)^2 + (z - z0)^2)")
    force.addPerParticleParameter("x0")
    force.addPerParticleParameter("y0")
    force.addPerParticleParameter("z0")
    force.addGlobalParameter("k", k)

    for i, atom in enumerate(topology.atoms()):
        if atom.name == "CA":
            pos = positions[i]
            force.addParticle(i, [pos.x, pos.y, pos.z])
    return force

# ✅ Updated gmx_grompp function with optional restraint logic for equilibration
def gmx_grompp(modeller, forcefield, mdp):
    method = mdp.get('nonbondedMethod', PME)
    cutoff = mdp.get('nonbondedCutoff', 1.0 * nanometer)
    constraints = mdp.get('constraints', HBonds)
    temperature = mdp.get('temperature', 300 * kelvin)
    friction = mdp.get('friction', 1 / picosecond)
    timestep = mdp.get('timestep', 0.002 * picoseconds)
    stage = mdp.get('stage', None)

    # Build system
    system = forcefield.createSystem(modeller.topology,
                                     nonbondedMethod=method,
                                     nonbondedCutoff=cutoff,
                                     constraints=constraints)

    # Apply C-alpha restraints if this is equilibration
    if stage == "eq":
        k = mdp.get("eq_restraint_force", 1000.0)
        restraint = build_ca_restraints(modeller.topology, modeller.positions, k)
        system.addForce(restraint)

    integrator = LangevinIntegrator(temperature, friction, timestep)

    try:
        platform = Platform.getPlatformByName('CUDA')
    except Exception:
        platform = Platform.getPlatformByName('CPU')

    simulation = Simulation(modeller.topology, system, integrator, platform)
    simulation.context.setPositions(modeller.positions)

    return simulation

# ✅ gmx_mdrun updated: stores final positions in modeller for all stages, writes trajectory
def gmx_mdrun(simulation, stage="em", steps=0, output_prefix=None,
              report_interval=1000, verbose=True, mdp=None, modeller=None):
    if output_prefix is None:
        output_prefix = stage

    if stage == "em":
        print("🔧 Minimizing energy...")
        tolerance = mdp.get('em_tolerance', 10.0) if mdp else 10.0
        max_iterations = mdp.get('em_max_iterations', 0) if mdp else 0

        if verbose:
            energy_before = simulation.context.getState(getEnergy=True).getPotentialEnergy()
            print(f"🔹 Initial potential energy: {energy_before}")

        simulation.minimizeEnergy(tolerance=tolerance, maxIterations=max_iterations)

        if verbose:
            energy_after = simulation.context.getState(getEnergy=True).getPotentialEnergy()
            print(f"🔹 Final potential energy: {energy_after}")

        positions = simulation.context.getState(getPositions=True).getPositions()
        PDBFile.writeFile(simulation.topology, positions, open(f"{output_prefix}.pdb", "w"))
        print(f"📦 Minimized structure saved to {output_prefix}.pdb")

        if modeller is not None:
            modeller.positions = positions

    elif stage in {"eq", "md"}:
        print(f"🚀 Running {stage.upper()} for {steps} steps...")

        if verbose:
            simulation.reporters.append(
                StateDataReporter(stdout, report_interval, step=True, time=True,
                                  temperature=True, potentialEnergy=True, progress=True,
                                  remainingTime=True, speed=True, totalSteps=steps, separator='\t')
            )

        # ✅ Save trajectory to DCD
        simulation.reporters.append(DCDReporter(f"{output_prefix}.dcd", report_interval))

        # Optionally also write the final frame as PDB
        simulation.reporters.append(PDBReporter(f"{output_prefix}_last.pdb", steps))

        simulation.step(steps)
        print(f"📦 {stage.upper()} trajectory saved to {output_prefix}.dcd")

        if modeller is not None:
            modeller.positions = simulation.context.getState(getPositions=True).getPositions()

    else:
        raise ValueError("Invalid stage. Choose 'em', 'eq', or 'md'.")

In [ ]:
#@title 📥 Load and preprocess protein structure from the PDB (adds missing atoms, hydrogens, etc.)
protein = load_protein("1BFF")

In [ ]:
#@title 👀 Visualize the prepared protein structure using py3Dmol (cartoon, water, ions)
vmd(protein)

In [ ]:
#@title 💧 Solvate Protein and Add Ions

# Select a ForceField
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

# Create Modeller and delete non-standard residues (like PO4)
system = gmx_pdb2gmx(protein, forcefield=forcefield)

# Add solvent and ions
system.addSolvent(forcefield, model='tip3p', padding=1.0*nanometers, ionicStrength=0.15*molar)

print("Solvated system has", system.topology.getNumAtoms(), "atoms")

In [ ]:
#@title 📝 MDP Parameters (GROMACS-style dictionary)
mdp_file = {
    'nonbondedMethod': PME,
    'nonbondedCutoff': 1.0 * nanometer,
    'constraints': HBonds,
    'temperature': 300 * kelvin,
    'friction': 1 / picosecond,
    'timestep': 0.002 * picoseconds,
    
    # Energy minimization settings
    'em_tolerance': 500.0,     # in kJ/mol/nm
    'em_max_iterations': 0      # 0 = no limit
}

In [ ]:
#@title 🔧 Prepare the simulation object from the Modeller, force field, and mdp settings
# This creates the System, Integrator, and Simulation objects (like GROMACS grompp)
em_tpr = gmx_grompp(system, forcefield, mdp_file)

# ⚙️ Run energy minimization using settings from the mdp file
# - Uses 'em_tolerance' and 'em_max_iterations' from mdp_file
# - Saves minimized structure as 'em.pdb'
# - Prints initial and final potential energy if verbose=True
gmx_mdrun(em_tpr, stage="em", mdp=mdp_file, verbose=True,  modeller=system)

In [ ]:
#@title 📝 MDP Parameters (GROMACS-style dictionary)
# Update mdp for equilibration
mdp_file.update({
    'stage': 'eq',
    'eq_restraint_force': 1000.0
})

In [ ]:
#@title 🌡️ Run equilibration with position restraints on Cα atoms
# - Sets 'stage' in mdp_file so gmx_grompp knows to apply Cα restraints
# - Applies harmonic restraints using 'eq_restraint_force' from mdp_file
# - Runs for 5000 steps (~10 ps if timestep = 2 fs)
# - Reports step, time, temperature, energy, speed, etc. if verbose=True


# 📦 Build the simulation object with the appropriate force field and position restraints
# Get minimized positions from the simulation
eq_tpr = gmx_grompp(system, forcefield, mdp_file)

# 🚀 Run equilibration with verbose logging
gmx_mdrun(eq_tpr, stage="eq", steps=2000, mdp=mdp_file, verbose=True, modeller=system, report_interval=100)

In [ ]:
#@title 🌡️ Run production Run!!!

mdp_file = {
    'nonbondedMethod': PME,
    'nonbondedCutoff': 1.0 * nanometer,
    'constraints': HBonds,
    'temperature': 300 * kelvin,
    'friction': 1 / picosecond,
    'timestep': 0.002 * picoseconds,
}

# 📌 Set stage to 'md' — this prevents restraints from being applied
mdp_file["stage"] = "md"

# Build the simulation (new integrator, no restraints):
md_tpr = gmx_grompp(system, forcefield, mdp_file)

# 🚀 Run equilibration with verbose logging
gmx_mdrun(md_tpr, stage="md", steps=2000, report_interval=100, mdp=mdp_file, verbose=True, modeller=system)

In [ ]:
#@title 🛠️ Convert DCD trajectory to multi-frame PDB for visualization
# - Uses the final structure ("md_last.pdb") as the topology
# - Reads coordinates from the DCD file ("md.dcd")
# - Writes all frames into a single multi-model PDB file ("md_trajectory.pdb")
convert_dcd_to_pdb("md_last.pdb", "md.dcd", "md_trajectory.pdb")

In [ ]:
#@title 👀 Visualize the trajectory
vmd_trajectory("md_trajectory.pdb")